# Capítulo 3

ÍNDICE
1. [RDDs vs. DataFrames](#RDDs-vs.-DataFrames)
2. [Formas de definir un esquema para un DataFrame](#Formas-de-definir-un-esquema-para-un-DataFrame)
3. [Mostrar esquema de un DF](#Mostrar-esquema-de-un-DF)
4. [Columnas y expresiones](#Columnas-y-expresiones)
5. [DataFrameReader y DataFrameWriter](#DataFrameReader-y-DataFrameWriter)
6. [Transformaciones y acciones](#Transformaciones-y-acciones)
7. [Cuestiones del libro](#Cuestiones-del-libro)
8. [Cuestiones adicionales](#Cuestiones-adicionales)

## RDDs vs. DataFrames
### RDD

In [0]:
data_rdd = sc.parallelize([('Jonny', 29), ('Martha', 53), ('Gaia', 34), ('Martha', 18), ('Takagi', 28)])
ages_rdd = (data_rdd
          .map(lambda x: (x[0], (x[1], 1)))
          .reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1]))
          .map(lambda x: (x[0], x[1][0]/x[1][1])))
ages_rdd.collect()

Out[18]: [('Martha', 35.5), ('Takagi', 28.0), ('Jonny', 29.0), ('Gaia', 34.0)]

### DataFrame

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg

spark = (SparkSession
        .builder
        .appName('AuthorsAges')
        .getOrCreate()
        )
data_df = spark.createDataFrame([('Jonny', 29), ('Martha', 53), ('Gaia', 34), ('Martha', 18), ('Takagi', 28)], ['name', 'age'])
avg_df = data_df.groupBy('name').agg(avg('age'))
avg_df.show()

+------+--------+
 name|avg(age)|
+------+--------+
 Jonny| 29.0|
Martha| 35.5|
 Gaia| 34.0|
Takagi| 28.0|
+------+--------+

Como podemos observar, al usar los DataFrames y las funciones de alto nivel que estos permiten, nuestro código es mucho más legible y fácil de entender, mantener y modificar.

## Formas de definir un esquema para un DataFrame
Los esquemas en Spark se pueden definir de dos formas:
### Programática

In [0]:
from pyspark.sql.types import *
schema = StructType([StructField('author', StringType(), False),
                    StructField('title', StringType(), False),
                    StructField('pages', IntegerType(), False)])

### Usando DDL

In [0]:
schema = 'author STRING, title STRING, pages INT'

> Ambas formas son equivalentes, aunque la forma que emplea DDL es mucho más sencilla.

## Mostrar esquema de un DF
Partiendo de los siguientes datos:

In [0]:
data = [[1, "Jules", "Damji", "https://tinyurl.1", "1/4/2016", 4535, ["twitter",
"LinkedIn"]],
        [2, "Brooke","Wenig", "https://tinyurl.2", "5/5/2018", 8908, ["twitter",
"LinkedIn"]],
        [3, "Denny", "Lee", "https://tinyurl.3", "6/7/2019", 7659, ["web",
"twitter", "FB", "LinkedIn"]],
        [4, "Tathagata", "Das", "https://tinyurl.4", "5/12/2018", 10568,
["twitter", "FB"]],
        [5, "Matei","Zaharia", "https://tinyurl.5", "5/14/2014", 40578, ["web",
"twitter", "FB", "LinkedIn"]],
        [6, "Reynold", "Xin", "https://tinyurl.6", "3/2/2015", 25568,
["twitter", "LinkedIn"]]
       ]

Y el siguiente esquema:

In [0]:
schema = "`Id` INT, `First` STRING, `Last` STRING, `Url` STRING, `Published` STRING, `Hits` INT, `Campaigns` ARRAY<STRING>"

Creamos un DataFrame y mostramos su contenido mediante el método *show()*.

In [0]:
blogs_df = spark.createDataFrame(data, schema)
blogs_df.show()

+---+---------+-------+-----------------+---------+-----+--------------------+
 Id| First| Last| Url|Published| Hits| Campaigns|
+---+---------+-------+-----------------+---------+-----+--------------------+
 1| Jules| Damji|https://tinyurl.1| 1/4/2016| 4535| [twitter, LinkedIn]|
 2| Brooke| Wenig|https://tinyurl.2| 5/5/2018| 8908| [twitter, LinkedIn]|
 3| Denny| Lee|https://tinyurl.3| 6/7/2019| 7659|[web, twitter, FB...|
 4|Tathagata| Das|https://tinyurl.4|5/12/2018|10568| [twitter, FB]|
 5| Matei|Zaharia|https://tinyurl.5|5/14/2014|40578|[web, twitter, FB...|
 6| Reynold| Xin|https://tinyurl.6| 3/2/2015|25568| [twitter, LinkedIn]|
+---+---------+-------+-----------------+---------+-----+--------------------+

Y por último, mostramos su esquema mediante la función *printSchema()* del DF.

In [0]:
print(blogs_df.printSchema())

root
-- Id: integer (nullable = true)
-- First: string (nullable = true)
-- Last: string (nullable = true)
-- Url: string (nullable = true)
-- Published: string (nullable = true)
-- Hits: integer (nullable = true)
-- Campaigns: array (nullable = true)
 |-- element: string (containsNull = true)

None

Además, podemos obtener la definición programática del esquema si accedemos a la propiedad *schema* del DF.

In [0]:
blogs_df.schema

Out[26]: StructType(List(StructField(Id,IntegerType,true),StructField(First,StringType,true),StructField(Last,StringType,true),StructField(Url,StringType,true),StructField(Published,StringType,true),StructField(Hits,IntegerType,true),StructField(Campaigns,ArrayType(StringType,true),true)))

## Columnas y expresiones
Podemos obtener el nombre de las columnas de un DF mediante el atributo *columns*

In [0]:
blogs_df.columns

Out[27]: ['Id', 'First', 'Last', 'Url', 'Published', 'Hits', 'Campaigns']

También podemos realizar expresiones lógicas o matemáticas sobre las columnas, especificándolas dentro de la función *expr()*:

In [0]:
import pyspark.sql.functions as F
blogs_df.select(F.expr('Hits * 2')).show(5)

+----------+
(Hits * 2)|
+----------+
 9070|
 17816|
 15318|
 21136|
 81156|
+----------+
only showing top 5 rows

El código anterior se puede reescribir computando los valores de las columnas mediante la función *col()*:

In [0]:
blogs_df.select(F.col('Hits')*2).show(5)

+----------+
(Hits * 2)|
+----------+
 9070|
 17816|
 15318|
 21136|
 81156|
+----------+
only showing top 5 rows

Podemos incluir estas columnas dentro de nuestro DF mediante la función *withColumn()* (aunque recordemos que los DF son inmutables, por lo que debemos crear uno nuevo)

In [0]:
big_hitters_blogs_df = blogs_df.withColumn('Big Hitters', F.col('Hits') > 10000)
big_hitters_blogs_df.show(5)

+---+---------+-------+-----------------+---------+-----+--------------------+-----------+
 Id| First| Last| Url|Published| Hits| Campaigns|Big Hitters|
+---+---------+-------+-----------------+---------+-----+--------------------+-----------+
 1| Jules| Damji|https://tinyurl.1| 1/4/2016| 4535| [twitter, LinkedIn]| false|
 2| Brooke| Wenig|https://tinyurl.2| 5/5/2018| 8908| [twitter, LinkedIn]| false|
 3| Denny| Lee|https://tinyurl.3| 6/7/2019| 7659|[web, twitter, FB...| false|
 4|Tathagata| Das|https://tinyurl.4|5/12/2018|10568| [twitter, FB]| true|
 5| Matei|Zaharia|https://tinyurl.5|5/14/2014|40578|[web, twitter, FB...| true|
+---+---------+-------+-----------------+---------+-----+--------------------+-----------+
only showing top 5 rows

## DataFrameReader y DataFrameWriter
### Reader
Para leer ficheros usamos la API DataFrameReader, accesible a través de *SparkSession.read*. De manera nativa, Spark puede leer los siguientes formatos: ```json, csv, parquet, orc, text, jdbc, libsvm```; aunque se puede ampliar para que trabaje con tipos de ficheros personalizados.

Leeremos el fichero *[sf-fire-calls.csv](https://github.com/databricks/LearningSparkV2/blob/master/chapter3/data/sf-fire-calls.csv)*, del que le inferiremos el esquema. Obtendremos el fichero del Bucket S3 público de los autores del libro (dbfs://databricks-datasets/learning-spark-v2/sf-fire/sf-fire-calls.csv)

Primero especificamos la ruta del fichero, y posteriormente su esquema, atendiendo a los nombres de los campos y sus tipos. Una vez construido el esquema, leemos el fichero con la función *csv()* del DataFrameReader. Además, le añadiremos las siguientes opciones:
- *header=True*: tomará los nombres de las columnas de la primera fila del csv.
- *schema*: tomará el esquema de nuestra definición.

Por útlimo, cacheamos el DF, ya que estaremos trabajando bastante sobre él.

In [0]:
fire_file = '/databricks-datasets/learning-spark-v2/sf-fire/sf-fire-calls.csv'
fire_schema = StructType([StructField('CallNumber', IntegerType(), True),
                     StructField('UnitID', StringType(), True),
                     StructField('IncidentNumber', IntegerType(), True),
                     StructField('CallType', StringType(), True),                  
                     StructField('CallDate', StringType(), True),      
                     StructField('WatchDate', StringType(), True),
                     StructField('CallFinalDisposition', StringType(), True),
                     StructField('AvailableDtTm', StringType(), True),
                     StructField('Address', StringType(), True),       
                     StructField('City', StringType(), True),       
                     StructField('Zipcode', IntegerType(), True),       
                     StructField('Battalion', StringType(), True),                 
                     StructField('StationArea', StringType(), True),       
                     StructField('Box', StringType(), True),       
                     StructField('OriginalPriority', StringType(), True),       
                     StructField('Priority', StringType(), True),       
                     StructField('FinalPriority', IntegerType(), True),       
                     StructField('ALSUnit', BooleanType(), True),       
                     StructField('CallTypeGroup', StringType(), True),
                     StructField('NumAlarms', IntegerType(), True),
                     StructField('UnitType', StringType(), True),
                     StructField('UnitSequenceInCallDispatch', IntegerType(), True),
                     StructField('FirePreventionDistrict', StringType(), True),
                     StructField('SupervisorDistrict', StringType(), True),
                     StructField('Neighborhood', StringType(), True),
                     StructField('Location', StringType(), True),
                     StructField('RowID', StringType(), True),
                     StructField('Delay', FloatType(), True)])
fire_df = spark.read.csv(fire_file, schema=fire_schema, header=True)
fire_df.cache()
fire_df.schema

Out[36]: StructType(List(StructField(CallNumber,IntegerType,true),StructField(UnitID,StringType,true),StructField(IncidentNumber,IntegerType,true),StructField(CallType,StringType,true),StructField(CallDate,StringType,true),StructField(WatchDate,StringType,true),StructField(CallFinalDisposition,StringType,true),StructField(AvailableDtTm,StringType,true),StructField(Address,StringType,true),StructField(City,StringType,true),StructField(Zipcode,IntegerType,true),StructField(Battalion,StringType,true),StructField(StationArea,StringType,true),StructField(Box,StringType,true),StructField(OriginalPriority,StringType,true),StructField(Priority,StringType,true),StructField(FinalPriority,IntegerType,true),StructField(ALSUnit,BooleanType,true),StructField(CallTypeGroup,StringType,true),StructField(NumAlarms,IntegerType,true),StructField(UnitType,StringType,true),StructField(UnitSequenceInCallDispatch,IntegerType,true),StructField(FirePreventionDistrict,StringType,true),StructField(SupervisorDistrict,StringType,true),StructField(Neighborhood,StringType,true),StructField(Location,StringType,true),StructField(RowID,StringType,true),StructField(Delay,FloatType,true)))

### Writer
De manera similar al caso anterior, para escribir ficheros empleamos la API DataFrameWriter, accesible desde el operador *DataFrame.write*. Debemos especificar el formato en el que queremos guardar el fichero y la ruta del mismo:

In [0]:
export_path = '/tmp/sf-fire-calls/'
fire_df.write.format('parquet').mode('overwrite').save(export_path)

In [0]:
%fs ls /tmp/sf-fire-calls/

path,name,size
dbfs:/tmp/sf-fire-calls/_SUCCESS,_SUCCESS,0
dbfs:/tmp/sf-fire-calls/_committed_7610182441254310909,_committed_7610182441254310909,924
dbfs:/tmp/sf-fire-calls/_started_1704619618890480202,_started_1704619618890480202,0
dbfs:/tmp/sf-fire-calls/_started_7610182441254310909,_started_7610182441254310909,0
dbfs:/tmp/sf-fire-calls/part-00000-tid-7610182441254310909-5591a2f1-619f-4592-8360-211676019adf-142-1-c000.snappy.parquet,part-00000-tid-7610182441254310909-5591a2f1-619f-4592-8360-211676019adf-142-1-c000.snappy.parquet,17447092
dbfs:/tmp/sf-fire-calls/part-00001-tid-7610182441254310909-5591a2f1-619f-4592-8360-211676019adf-143-1-c000.snappy.parquet,part-00001-tid-7610182441254310909-5591a2f1-619f-4592-8360-211676019adf-143-1-c000.snappy.parquet,19410414
dbfs:/tmp/sf-fire-calls/part-00002-tid-7610182441254310909-5591a2f1-619f-4592-8360-211676019adf-144-1-c000.snappy.parquet,part-00002-tid-7610182441254310909-5591a2f1-619f-4592-8360-211676019adf-144-1-c000.snappy.parquet,16679981
dbfs:/tmp/sf-fire-calls/part-00003-tid-7610182441254310909-5591a2f1-619f-4592-8360-211676019adf-145-1-c000.snappy.parquet,part-00003-tid-7610182441254310909-5591a2f1-619f-4592-8360-211676019adf-145-1-c000.snappy.parquet,17537285
dbfs:/tmp/sf-fire-calls/part-00004-tid-7610182441254310909-5591a2f1-619f-4592-8360-211676019adf-146-1-c000.snappy.parquet,part-00004-tid-7610182441254310909-5591a2f1-619f-4592-8360-211676019adf-146-1-c000.snappy.parquet,18908941
dbfs:/tmp/sf-fire-calls/part-00005-tid-7610182441254310909-5591a2f1-619f-4592-8360-211676019adf-147-1-c000.snappy.parquet,part-00005-tid-7610182441254310909-5591a2f1-619f-4592-8360-211676019adf-147-1-c000.snappy.parquet,19043468


También podemos guardarlo como una tabla, de modo que se registraran sus metadatos en el metastore de Hive:

In [0]:
table_name = 'fire_calls'
fire_df.write.format('parquet').mode('overwrite').saveAsTable(table_name)

In [0]:
%sql
CACHE TABLE fire_calls

In [0]:
%sql
SELECT * FROM fire_calls LIMIT 5

CallNumber,UnitID,IncidentNumber,CallType,CallDate,WatchDate,CallFinalDisposition,AvailableDtTm,Address,City,Zipcode,Battalion,StationArea,Box,OriginalPriority,Priority,FinalPriority,ALSUnit,CallTypeGroup,NumAlarms,UnitType,UnitSequenceInCallDispatch,FirePreventionDistrict,SupervisorDistrict,Neighborhood,Location,RowID,Delay
61160285,E22,6032206,Medical Incident,04/26/2006,04/26/2006,Other,null,1400 Block of 7TH AVE,SF,94122,B08,22,7331,3,3,3,false,null,1,ENGINE,1,8,5,Inner Sunset,"(37.7613340585607, -122.464046025734)",061160285-E22,3.0333333
61160287,93,6032207,Medical Incident,04/26/2006,04/26/2006,Code 2 Transport,04/26/2006 02:32:37 PM,400 Block of BROADWAY,SF,94133,B01,13,1231,E,E,3,true,null,1,MEDIC,2,1,3,North Beach,"(37.7981219270604, -122.405087523335)",061160287-93,3.1166666
61160287,E13,6032207,Medical Incident,04/26/2006,04/26/2006,Other,04/26/2006 01:40:18 PM,400 Block of BROADWAY,SF,94133,B01,13,1231,E,E,3,false,null,1,ENGINE,1,1,3,North Beach,"(37.7981219270604, -122.405087523335)",061160287-E13,6.5666666
61160287,RC4,6032207,Medical Incident,04/26/2006,04/26/2006,Other,04/26/2006 01:47:22 PM,400 Block of BROADWAY,SF,94133,B01,13,1231,E,E,3,true,null,1,RESCUE CAPTAIN,3,1,3,North Beach,"(37.7981219270604, -122.405087523335)",061160287-RC4,6.0333333
61160293,B02,6032209,Alarms,04/26/2006,04/26/2006,Other,04/26/2006 01:31:30 PM,0 Block of SANCHEZ ST,SF,94114,B05,06,5131,3,3,3,false,null,1,CHIEF,3,2,8,Castro/Upper Market,"(37.7684622459015, -122.431204878453)",061160293-B02,3.1333334


## Transformaciones y acciones
Ahora que tenemos nuestro DF, examinaremos los datos que contiene.
### Proyecciones
Una proyección es una acotación de los datos que devuelve únicamente las filas que cumpla unas ciertas condiciones, establecidas mediante filtros. Para ello, emplearemos las funciones ```select()```, que crean las proyecciones; y ```filter()``` o ```where()``` que establecen los filtros.

Obtenemos los campos 'IncidentNumber, 'AvailableDtTm' y 'CallType' de los registros cuyo *'CallType'* es *'Medical Incident'*:

In [0]:
medical_fire_df = (fire_df
                  .select('IncidentNumber', 'AvailableDtTm', 'CallType')
                  .where(F.col('CallType') == 'Medical Incident'))
medical_fire_df.show(5)

+--------------+--------------------+----------------+
IncidentNumber| AvailableDtTm| CallType|
+--------------+--------------------+----------------+
 2003234|01/11/2002 01:58:...|Medical Incident|
 2003233|01/11/2002 02:10:...|Medical Incident|
 2003236|01/11/2002 02:27:...|Medical Incident|
 2003238|01/11/2002 02:28:...|Medical Incident|
 2003240|01/11/2002 02:17:...|Medical Incident|
+--------------+--------------------+----------------+
only showing top 5 rows

Contamos el número de tipos de 'CallType' distintos, quitando previamente los nulos:

In [0]:
(fire_df
    .select('CallType')
    .where(F.col('CallType').isNotNull())
    .agg(F.countDistinct('CallType').alias('DistinctCallTypes'))
    .show())

+-----------------+
DistinctCallTypes|
+-----------------+
 32|
+-----------------+

Mostramos los tipos distintos de 'CallType':

In [0]:
(fire_df
    .select('CallType')
    .where(F.col('CallType').isNotNull())
    .distinct()
    .show(10, False))

+-----------------------------+
CallType |
+-----------------------------+
Alarms |
Odor (Strange / Unknown) |
Citizen Assist / Service Call|
Vehicle Fire |
Other |
Outside Fire |
Electrical Hazard |
Structure Fire |
Medical Incident |
Fuel Spill |
+-----------------------------+
only showing top 10 rows

### Renombrar, añadir y quitar columnas
Podemos renombrar columnas con la función ```withColumnRenamed(nombreActual, nombreNuevo)```, que nos devolverá un nuevo DF con la columna renombrada.

In [0]:
(fire_df
    .withColumnRenamed('Delay', 'DelayInMins')
    .select('DelayInMins')
    .where(F.col('DelayInMins') > 5)
    .show(5)
)

+-----------+
DelayInMins|
+-----------+
 5.233333|
 6.9333334|
 6.116667|
 7.85|
 77.333336|
+-----------+
only showing top 5 rows

Si queremos añadir columnas, emplearemos la función ```withColumn(nombreColumna, valor)```. Por otro lado, para remover una columna, usaremos ```drop(nombreColumna)```. Ambas nos devolverán un nuevo DF con las columnas añadidas/eliminadas.

Por ejemplo, en este DF, las columnas que expresan fechas tienen el tipo String, por lo que no se les puede efectuar operaciones de fechas. Para cambiarles el tipo, crearemos un nuevo DF con una nueva columna con el tipo *timestamp* y eliminaremos la columna equivalente de tipo String.

In [0]:
fire_ts_df = (fire_df
    .withColumn('IncidentDate', F.to_timestamp(F.col('CallDate'), 'MM/dd/yyyy'))
    .drop('CallDate')
    .withColumn('OnWatchDate', F.to_timestamp(F.col('WatchDate'), 'MM/dd/yyyy'))
    .drop('WatchDate')
    .withColumn('AvailableDtTS', F.to_timestamp(F.col('AvailableDtTm'), 'MM/dd/yyyy hh:mm:ss a'))
    .drop('AvailableDtTm')
)
fire_ts_df.cache()
(fire_ts_df
 .select('IncidentDate', 'OnWatchDate', 'AvailableDtTS')
 .show(5, False)
)

+-------------------+-------------------+-------------------+
IncidentDate |OnWatchDate |AvailableDtTS |
+-------------------+-------------------+-------------------+
2002-01-11 00:00:00|2002-01-10 00:00:00|2002-01-11 01:58:43|
2002-01-11 00:00:00|2002-01-10 00:00:00|2002-01-11 02:10:17|
2002-01-11 00:00:00|2002-01-10 00:00:00|2002-01-11 01:47:00|
2002-01-11 00:00:00|2002-01-10 00:00:00|2002-01-11 01:51:54|
2002-01-11 00:00:00|2002-01-10 00:00:00|2002-01-11 01:47:00|
+-------------------+-------------------+-------------------+
only showing top 5 rows

A continuación, podremos ejecutar sobre estas columnas las funciones de pyspark.sql.functions, como year(), month() o day(). Por ejemplo, listaremos los incidentes agrupados por año y los ordenaremos de manera descedente por número de incidentes:

In [0]:
(fire_ts_df
.select(F.year('IncidentDate').alias('Year'), 'IncidentNumber')
.groupBy('Year')
.agg(F.count('IncidentNumber').alias('CountOfIncidents'))
.orderBy('CountOfIncidents', ascending=False)
.show()
)

+----+----------------+
Year|CountOfIncidents|
+----+----------------+
2017| 301449|
2016| 292526|
2015| 285281|
2014| 268074|
2018| 254602|
2013| 248796|
2011| 242121|
2012| 241714|
2010| 228567|
2008| 221652|
2009| 217800|
2003| 214503|
2004| 211056|
2007| 208250|
2006| 204616|
2005| 204569|
2002| 201575|
2001| 194309|
2000| 139200|
+----+----------------+

## Cuestiones del libro
En este apartado, resolveremos las cuestiones propuestas en la página 68 del libro *Learing Spark 2nd Edition*, dentro del apartado *End-to-End DataFrame Example*.
1. ¿Cuáles fueron los tipos diferentes de llamadas que hubo en 2018?

A partir del DF con las fechas como tipo timestamp (*fire_ts_df*), filtraremos las llamadas de 2018 cuyo 'CallType' no sea nulo y obtendremos los distintos tipos de llamadas:

In [0]:
(fire_ts_df
 .select('CallType')
 .where(F.col('CallType').isNotNull())
 .where(F.year('IncidentDate') == 2018)
 .distinct()
 .show(truncate=False)
)

+--------------------------------------------+
CallType |
+--------------------------------------------+
Elevator / Escalator Rescue |
Alarms |
Odor (Strange / Unknown) |
Citizen Assist / Service Call |
HazMat |
Explosion |
Vehicle Fire |
Extrication / Entrapped (Machinery, Vehicle)|
Other |
Outside Fire |
Traffic Collision |
Assist Police |
Gas Leak (Natural and LP Gases) |
Water Rescue |
Electrical Hazard |
Structure Fire |
Industrial Accidents |
Medical Incident |
Fuel Spill |
Smoke Investigation (Outside) |
+--------------------------------------------+
only showing top 20 rows

2. ¿En qué meses de 2018 se registró un mayor número de llamadas?

Filtramos a las llamadas de 2018, agregamos el número de llamadas por mes y ordenamos de manera descendente por número de llamadas.

In [0]:
(fire_ts_df
 .select(F.month('IncidentDate').alias('Month'), 'IncidentNumber')
 .where(F.year('IncidentDate') == 2018)
 .groupBy('Month')
 .agg(F.count('IncidentNumber').alias('NumOfCalls'))
 .orderBy('NumOfCalls', ascending=False)
 .show(3, truncate=False)
)

+-----+----------+
Month|NumOfCalls|
+-----+----------+
1 |26148 |
3 |25755 |
10 |25606 |
+-----+----------+
only showing top 3 rows

3. ¿Qué barrio de San Francisco generó más llamadas en 2018?

La manera de proceder es igual que en punto anterior, cambiando la agrupación por mes a agrupación por 'Neighborhood' y limitando el resultado a 1.

In [0]:
(fire_ts_df
 .select('Neighborhood', 'IncidentNumber')
 .where(F.year('IncidentDate') == 2018)
 .groupBy('Neighborhood')
 .agg(F.count('IncidentNumber').alias('NumOfCalls'))
 .orderBy('NumOfCalls', ascending=False)
 .show(1, truncate=False)
)

+------------+----------+
Neighborhood|NumOfCalls|
+------------+----------+
Tenderloin |35557 |
+------------+----------+
only showing top 1 row

4. ¿Qué barrios tuvieron los peores tiempos de respuesta a las llamadas en 2018?

Para este caso, en lugar de agregar el número de llamadas, agregaremos la media de los minutos de 'Delay'.

In [0]:
(fire_ts_df
 .select('Neighborhood', 'Delay')
 .where(F.year('IncidentDate') == 2018)
 .groupBy('Neighborhood')
 .agg(F.avg('Delay').alias('AvgDelay'))
 .orderBy('AvgDelay', ascending=False)
 .show(5, truncate=False)
)

+---------------+-----------------+
Neighborhood |AvgDelay |
+---------------+-----------------+
Treasure Island|5.514422900772491|
Lincoln Park |5.50080411449859 |
Presidio |4.898821295123589|
Chinatown |4.785041692726091|
Inner Sunset |4.387634401598253|
+---------------+-----------------+
only showing top 5 rows

5. ¿En qué semana del año 2018 hubo más llamadas?

Este caso es igual al del punto 2, pero en lugar de por meses, por semanas del año.

In [0]:
(fire_ts_df
 .select(F.weekofyear('IncidentDate').alias('WeekOfYear'), 'IncidentNumber')
 .where(F.year('IncidentDate') == 2018)
 .groupBy('WeekOfYear')
 .agg(F.count('IncidentNumber').alias('NumOfCalls'))
 .orderBy('NumOfCalls', ascending=False)
 .show(1, truncate=False)
)

+----------+----------+
WeekOfYear|NumOfCalls|
+----------+----------+
1 |6401 |
+----------+----------+
only showing top 1 row

6. ¿Existe relación entre el barrio, el código postal y el número de llamadas?

No sé cómo dar respuesta a esta pregunta.

In [0]:
(fire_ts_df
 .select('ZipCode', 'Neighborhood', 'IncidentNumber')
 .groupBy('Neighborhood', 'ZipCode')
 .agg(F.count('IncidentNumber').alias('NumOfCalls'))
 .orderBy('NumOfCalls', ascending=False)
 .show()
)

+--------------------+-------+----------+
 Neighborhood|ZipCode|NumOfCalls|
+--------------------+-------+----------+
 Tenderloin| 94102| 427108|
 South of Market| 94103| 343420|
 Mission| 94110| 260184|
Bayview Hunters P...| 94124| 226886|
 Mission| 94103| 136172|
 Tenderloin| 94109| 132605|
 Outer Richmond| 94121| 105242|
Financial Distric...| 94105| 104501|
 Castro/Upper Market| 94114| 99755|
 Nob Hill| 94109| 99390|
 Western Addition| 94115| 97744|
 North Beach| 94133| 93799|
 Sunset/Parkside| 94122| 87284|
 Marina| 94123| 82589|
 Bernal Heights| 94110| 80263|
 Excelsior| 94112| 78370|
 Sunset/Parkside| 94116| 74496|
 Hayes Valley| 94102| 68152|
 Haight Ashbury| 94117| 68037|
 Outer Mission| 94112| 65126|
+--------------------+-------+----------+
only showing top 20 rows

7. ¿Cómo podemos almacenar esta información en ficheros Parquet o tablas SQL y leerlos de nuevo?

### Parquet

Comenzamos guardando nuestro DF en formato Parquet:

In [0]:
fire_ts_df.write.format('parquet').mode('overwrite').save('/tmp/fire-ts-parquet')

Y lo leemos igualmente en formato Parquet:

In [0]:
new_fire_ts_df = spark.read.format('parquet').load('/tmp/fire-ts-parquet/')
new_fire_ts_df.show(1)

+----------+------+--------------+----------------+--------------------+--------------------+----+-------+---------+-----------+----+----------------+--------+-------------+-------+-------------+---------+--------+--------------------------+----------------------+------------------+------------+--------------------+-------------+---------+-------------------+-------------------+-------------+
CallNumber|UnitID|IncidentNumber| CallType|CallFinalDisposition| Address|City|Zipcode|Battalion|StationArea| Box|OriginalPriority|Priority|FinalPriority|ALSUnit|CallTypeGroup|NumAlarms|UnitType|UnitSequenceInCallDispatch|FirePreventionDistrict|SupervisorDistrict|Neighborhood| Location| RowID| Delay| IncidentDate| OnWatchDate|AvailableDtTS|
+----------+------+--------------+----------------+--------------------+--------------------+----+-------+---------+-----------+----+----------------+--------+-------------+-------+-------------+---------+--------+--------------------------+----------------------+------------------+------------+--------------------+-------------+---------+-------------------+-------------------+-------------+
 61160285| E22| 6032206|Medical Incident| Other|1400 Block of 7TH...| SF| 94122| B08| 22|7331| 3| 3| 3| false| null| 1| ENGINE| 1| 8| 5|Inner Sunset|(37.7613340585607...|061160285-E22|3.0333333|2006-04-26 00:00:00|2006-04-26 00:00:00| null|
+----------+------+--------------+----------------+--------------------+--------------------+----+-------+---------+-----------+----+----------------+--------+-------------+-------+-------------+---------+--------+--------------------------+----------------------+------------------+------------+--------------------+-------------+---------+-------------------+-------------------+-------------+
only showing top 1 row

### Tabla Parquet SQL

Igualmente, procedemos guardando el DF como una tabla en formato Parquet:

In [0]:
fire_ts_df.write.format('parquet').mode('overwrite').saveAsTable('FireServiceCalls')

--------------------------------------------------------------------------- 
 AnalysisException Traceback (most recent call last)
 <command-2969087339106091> in <module> 
 ----> 1 fire_ts_df . write . format ( 'parquet' ) . mode ( 'overwrite' ) . saveAsTable ( 'FireServiceCalls' ) 

 /databricks/spark/python/pyspark/sql/readwriter.py in saveAsTable (self, name, format, mode, partitionBy, **options) 
 866 if format is not None : 
 867 self . format ( format ) 
 --> 868 self . _jwrite . saveAsTable ( name ) 
 869 
 870 @ since ( 1.4 ) 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1303 answer = self . gateway_client . send_command ( command ) 
 1304 return_value = get_return_value(
 -> 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 
 1307 for temp_arg in temp_args : 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 131 # Hide where the exception came from that shows a non-Pythonic 
 132 # JVM exception message. 
 --> 133 raise_from ( converted ) 
 134 else : 
 135 raise 

 /databricks/spark/python/pyspark/sql/utils.py in raise_from (e) 

 AnalysisException : Can not create the managed table('`FireServiceCalls`'). The associated location('dbfs:/user/hive/warehouse/fireservicecalls') already exists.;

Posteriormente, lo leemos usando el SparkSession:

In [0]:
spark.table('FireServiceCalls').show(1)

--------------------------------------------------------------------------- 
 AnalysisException Traceback (most recent call last)
 <command-2969087339106093> in <module> 
 ----> 1 spark . table ( 'FireServiceCalls' ) . show ( 1 ) 

 /databricks/spark/python/pyspark/sql/session.py in table (self, tableName) 
 720 True 
 721 """
 --> 722 return DataFrame ( self . _jsparkSession . table ( tableName ) , self . _wrapped ) 
 723 
 724 @ property

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1303 answer = self . gateway_client . send_command ( command ) 
 1304 return_value = get_return_value(
 -> 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 
 1307 for temp_arg in temp_args : 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 131 # Hide where the exception came from that shows a non-Pythonic 
 132 # JVM exception message. 
 --> 133 raise_from ( converted ) 
 134 else : 
 135 raise 

 /databricks/spark/python/pyspark/sql/utils.py in raise_from (e) 

 AnalysisException : Table or view not found: FireServiceCalls;;
'UnresolvedRelation [FireServiceCalls]

## Cuestiones adicionales

Estas cuestiones se encuentran en un PDF de ejercicios extra a realizar después de cada capítulo.

1\. **Realizar todos los ejercicios propuestos del libro ✓**

2\. **Leer el CSV de ejemplo del Capítulo 2 y obtener la estructura del esquema dado por defecto.**

In [0]:
mnm_df_file = '/databricks-datasets/learning-spark-v2/mnm_dataset.csv'
mnm_df = spark.read.csv(mnm_df_file, inferSchema=True, header=True)
mnm_df.schema

Out[60]: StructType(List(StructField(State,StringType,true),StructField(Color,StringType,true),StructField(Count,IntegerType,true)))

3\. **Cuando se define un schema al definir un campo, por ejemplo, ```StructField('Delay', FloatType(), True)```, ¿qué significa el último parámetro Boolean?**

Este parámetro indica si los registros pueden contener un valor ```null``` en ese campo.

4\. **Dataset vs DataFrame (Scala). ¿En qué se diferencian a nivel de código?**

Esta cuestión concierne a Scala, por lo que se resolverá en el notebook de Scala.

5\. **Utilizando el mismo ejemplo utilizado en el capítulo para guardar en parquet y guardar los datos en los formatos:**
 1. *JSON*
 2. *CSV (dándole otro nombre para evitar sobrescribir el fichero origen)*
 3. *AVRO*

1\. JSON

In [0]:
fire_ts_df.write.format('json').mode('overwrite').save('/tmp/fire-ts-json')

2\. CSV

In [0]:
fire_ts_df.write.format('csv').mode('overwrite').save('/tmp/fire-ts-csv')

3\. AVRO

In [0]:
fire_ts_df.write.format('avro').mode('overwrite').save('/tmp/fire-ts-avro')

In [0]:
%fs ls /tmp/fire-ts-avro/

path,name,size
dbfs:/tmp/fire-ts-avro/_SUCCESS,_SUCCESS,0
dbfs:/tmp/fire-ts-avro/_committed_4663910036655479192,_committed_4663910036655479192,834
dbfs:/tmp/fire-ts-avro/_started_4663910036655479192,_started_4663910036655479192,0
dbfs:/tmp/fire-ts-avro/part-00000-tid-4663910036655479192-4ab45ccb-9868-4932-804e-9e76afd9b26a-294-1-c000.avro,part-00000-tid-4663910036655479192-4ab45ccb-9868-4932-804e-9e76afd9b26a-294-1-c000.avro,36328699
dbfs:/tmp/fire-ts-avro/part-00001-tid-4663910036655479192-4ab45ccb-9868-4932-804e-9e76afd9b26a-295-1-c000.avro,part-00001-tid-4663910036655479192-4ab45ccb-9868-4932-804e-9e76afd9b26a-295-1-c000.avro,37105831
dbfs:/tmp/fire-ts-avro/part-00002-tid-4663910036655479192-4ab45ccb-9868-4932-804e-9e76afd9b26a-296-1-c000.avro,part-00002-tid-4663910036655479192-4ab45ccb-9868-4932-804e-9e76afd9b26a-296-1-c000.avro,35865336
dbfs:/tmp/fire-ts-avro/part-00003-tid-4663910036655479192-4ab45ccb-9868-4932-804e-9e76afd9b26a-297-1-c000.avro,part-00003-tid-4663910036655479192-4ab45ccb-9868-4932-804e-9e76afd9b26a-297-1-c000.avro,33827100
dbfs:/tmp/fire-ts-avro/part-00004-tid-4663910036655479192-4ab45ccb-9868-4932-804e-9e76afd9b26a-298-1-c000.avro,part-00004-tid-4663910036655479192-4ab45ccb-9868-4932-804e-9e76afd9b26a-298-1-c000.avro,34287073
dbfs:/tmp/fire-ts-avro/part-00005-tid-4663910036655479192-4ab45ccb-9868-4932-804e-9e76afd9b26a-299-1-c000.avro,part-00005-tid-4663910036655479192-4ab45ccb-9868-4932-804e-9e76afd9b26a-299-1-c000.avro,36106389
dbfs:/tmp/fire-ts-avro/part-00006-tid-4663910036655479192-4ab45ccb-9868-4932-804e-9e76afd9b26a-300-1-c000.avro,part-00006-tid-4663910036655479192-4ab45ccb-9868-4932-804e-9e76afd9b26a-300-1-c000.avro,36411441


6\. Revisar el número de ficheros
creados al guardar los ficheros, revisar su contenido para constatar como se guardan.

I. ¿A qué se debe que hayan más de un fichero?

Se debe a que se crea un fichero por cada partición que emplea Spark.

II. ¿Cómo se obtine el número de particiones de un DataFrame?

Se obtiene mediante el método ```getNumPartitions()``` que se aplica sobre la propiedad ```rdd``` del DF.

In [0]:
fire_ts_df.rdd.getNumPartitions()

Out[62]: 9

III. ¿Qué formas existen para modificar el número de particiones de un DataFrame?

Para modificar el número de particiones, creamos un nuevo DF a partir de la función ```repartition(numParticiones)``` del DF.

IV. Llevar a cabo el ejemplo modificando el número de particiones a 1 y revisar de nuevo el/los ficheros guardados.

In [0]:
repartitioned_fire_ts_df = fire_ts_df.repartition(1)
repartitioned_fire_ts_df.write.format('csv').mode('overwrite').save('/tmp/rep-fire-ts-csv')

In [0]:
%fs ls /tmp/rep-fire-ts-csv

path,name,size
dbfs:/tmp/rep-fire-ts-csv/_SUCCESS,_SUCCESS,0
dbfs:/tmp/rep-fire-ts-csv/_committed_6643270210129314217,_committed_6643270210129314217,113
dbfs:/tmp/rep-fire-ts-csv/_started_6643270210129314217,_started_6643270210129314217,0
dbfs:/tmp/rep-fire-ts-csv/part-00000-tid-6643270210129314217-99228e03-c768-43f7-871c-32d80c25cfbe-314-1-c000.csv,part-00000-tid-6643270210129314217-99228e03-c768-43f7-871c-32d80c25cfbe-314-1-c000.csv,1244038882
